In [49]:
import numpy as np
import sys
from casadi import *
import matplotlib.pyplot as plt
import math
# Import do_mpc package:
import do_mpc
T=1000
r=np.array([0,0,-1])

In [50]:
model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)

In [51]:
r=model.set_variable('_x','linear',(3))
v=model.set_variable('_x','dlinear',(3))
m=model.set_variable('_x','mass',(1))
q=model.set_variable('_x','attitude',(4))
omega=model.set_variable('_x','omega',(3))
g=np.array([0,0,-9.81])
d=model.set_variable('_u','tvc',(3))
F_B=d*T
F_N=SX.sym('force_in',3)
x,y,z=vertsplit(F_B)
F_N=SX.sym('force_in',3)
qw,qx,qy,qz=vertsplit(q)
F_N[0] = x*(qx*qx+qw*qw-qy*qy- qz*qz) + y*(2*qx*qy- 2*qw*qz) + z*(2*qx*qz+ 2*qw*qy)
F_N[1] = x*(2*qw*qz + 2*qx*qy) + y*(qw*qw - qx*qx+ qy*qy - qz*qz)+ z*(-2*qw*qx+ 2*qy*qz)
F_N[2] = x*(-2*qw*qy+ 2*qx*qz) + y*(2*qw*qx+ 2*qy*qz)+ z*(qw*qw - qx*qx- qy*qy+ qz*qz)
J = SX.sym('moi', 3, 3)
J[0, 0] = 10
J[0, 1] = 0
J[0, 2] = 0
J[1, 0] = 0
J[1, 1] = 10
J[1, 2] = 0
J[2,0]=0
J[2, 1] = 0
J[2, 2] = 10
op=SX.sym('op',3,3)
op[0,0]=0
op[0,1]=-omega[2]
op[0,2]=omega[1]
op[1,0]=omega[2]
op[1,1]=0
op[1,2]=-omega[0]
op[2,0]=-omega[1]
op[2,1]=omega[0]
op[2,2]=0
L_B=cross(r,F_B)
oh=SX.sym('oh',4,4)
oh[0,0]=0
oh[0,1]=-omega[0]
oh[0,2]=-omega[1]
oh[0,3]=-omega[2]
oh[1,0]=omega[0]
oh[1,1]=0
oh[1,2]=omega[2]
oh[1,3]=-omega[1]
oh[2,0]=omega[1]
oh[2,1]=-omega[2]
oh[2,2]=0
oh[2,3]=omega[0]
oh[3,0]=omega[2]
oh[3,1]=omega[1]
oh[3,2]=-omega[0]
oh[3,3]=0
oh
J_inv=inv(J)
domega=-J_inv@op@J@omega+L_B
dq=(oh@q)/2
dv=(F_N/m)+g
dm=(sqrt(F_B[0]**2+F_B[1]**2+F_B[2]**2))/100 # 100 is I_sp*g_ref

In [52]:
# M = 1 #Mach Number
# N = 1 #number of Fins 
# At = 1 #Planform area of one exposed fin 
# Ar = 1 #reference Area 
# S = 1 #fin semi span
# rl = 1 #fin leading edge radius 
# tl = 1 #leading edge sweep angle 
# Abf = 1 #base area of one fin 
# lc = 1 #midchord line sweep angle
# cr = 1 #fin root chord
# hr = 1 #root fin trailing edge thickness 
# tr = 1 #root fin thickness 
# Awb = 1 #wetted area of body
# fb = 1 #body and nose fineness ratio 
# Ab = 1 #base area at tail 
# Aw = 1 #total wetted area 
# Re=1 #Unknown
# Abn = 1 #Nosecone Base Area

# Af = 1 #total area of one fin 
# ar = 1 #aspect ratio
# rt = 1 #body radius at tail 
# s = 1 #exposed fin semi span 
# lc = 1 #midchord line sweep angle  

# ## Calculation for DragCoeff   
# Cf = 1.328/((Re)**0.5)
# Cfc = Cf*(1-(0.12*M*M))
# Cfb = 2*Cf*(cr/hr)
# Kdbt = ((1+((18*Cfc*(tr/hr)**2)/(N*(1-0.52)*(Abf/Ar)))) * ((0.135*N*(Abf/Ar))/(Cfc**(1/3)))) + ((math.cos(lc))**2)
# del_Cd = ((1-(M**2))**-0.417)-1
# Kdp = 1 + (((6.82*Awb*Cfc*((fn+0.7)**1.29))/((fb**3)*Ar))**(5/3))
# Kdbb = 1 +  (1/((6.38 + 39.7*(hr/cr))(Cfc*(Aw/Ab))))
# CDft = 2*N*Cfc*(At/Ar)
# CDlt = (2*N*S*rl*(math.cos(tl)**2)*del_Cd)/Ar
# CDbt = (0.135*N*Abf)/(Ar*(Cfb**(1/3))*(Kdbt-M*M*(math.cos(lc))**2)**0.5)
# CDtt = (4*N*Cfc) * (At/At) * ((tr/cr)*(math.cos(lc)**2) + ((30 * (tr/cr)**4 * math.cos(lc)**2) / ((K - M*M*(math.cos(lc)**2))**1.5)))       
# Cdtt = CDft + CDlt + CDbt + CDtt
# CDp = (6*Awb*Cfc) / (((fb)**3) * Ar * ((Kdp-(M**2))**0.6))
# CDbb = (0.29*(Ab / Ar)) / ((Cfc*(Aw/Ab)*(Kdbb-(M**2)))**0.5)
# Cdtb = CDp + CDbb
# Cd = Cdtt + Cdtb 

# ##Calculation for Normal Coeff
# B = (1-(M**2))**0.5
# pi = 3.141592
# tou = (s + rt)/rt
# Cnab = 2 * (Abn / Ar)
# Cnat = (N*pi*ar*(Af/Ar))/(2+(4+((B*ar)/(math.cos(lc)))**2)**0.5)
# Ktb = (2/(pi*((1-(1/tou))**2)))*(((1+(1/(tou**2)))*(0.5*m.atan(0.5*(tou-(1/tou)))))-((1/(tou**2))*(tou - (t/tou)+ 2*m.atan(1/tou))))
# Cnatb = Cnat * Ktb 
# Kbt = (((1-(1/(tou**2)))/(1-(t/tou)))**2) - Ktb 
# Cnabt = Cnat * Kbt 
# Cna = Cnab + Cnatb + Cnabt 

In [53]:
model.set_rhs('attitude',dq)
model.set_rhs('omega',domega)
model.set_rhs('dlinear',dv)
model.set_rhs('linear',v)
model.set_rhs('mass',dm)

In [54]:
model.setup()

In [55]:
simulator = do_mpc.simulator.Simulator(model)

In [56]:
params_simulator = {
    'integration_tool': 'cvodes',
    'abstol': 1e-10,
    'reltol': 1e-10,
    't_step': 0.005
}

simulator.set_param(**params_simulator)
simulator.setup()

In [57]:
estimator = do_mpc.estimator.StateFeedback(model)

In [58]:
x0=np.array([0,0,0,0,0,0,10,0,0,0,1,0,0,0]).reshape(-1,1)
simulator.x0 = x0
estimator.x0 = x0

In [72]:
data=[]
time=[]
for k in range(500):
    u0=np.array([[0],[0],[1]])
    y_next = simulator.make_step(u0)
    x0 = estimator.make_step(y_next)
    data.append(x0)
    time.append(k)

In [82]:
data=[x[2] for x in data]
data

[array([158.31478807]),
 array([158.81924698]),
 array([159.3241729]),
 array([159.8295648]),
 array([160.33542168]),
 array([160.84174253]),
 array([161.34852635]),
 array([161.85577213]),
 array([162.36347888]),
 array([162.87164559]),
 array([163.38027128]),
 array([163.88935496]),
 array([164.39889563]),
 array([164.90889232]),
 array([165.41934404]),
 array([165.93024981]),
 array([166.44160865]),
 array([166.95341959]),
 array([167.46568167]),
 array([167.9783939]),
 array([168.49155533]),
 array([169.00516499]),
 array([169.51922192]),
 array([170.03372517]),
 array([170.54867377]),
 array([171.06406678]),
 array([171.57990324]),
 array([172.09618222]),
 array([172.61290275]),
 array([173.13006391]),
 array([173.64766475]),
 array([174.16570434]),
 array([174.68418173]),
 array([175.203096]),
 array([175.72244623]),
 array([176.24223147]),
 array([176.76245082]),
 array([177.28310334]),
 array([177.80418812]),
 array([178.32570423]),
 array([178.84765078]),
 array([179.37002684]